# Przykładowy Agent Rezerwacji Hotelu i Lotów

To rozwiązanie pomoże Ci zarezerwować bilety lotnicze i hotel. Scenariusz to podróż z Londynu Heathrow LHR 20 lutego 2024 do Nowego Jorku JFK z powrotem 27 lutego 2025, w klasie ekonomicznej, tylko British Airways. Chcę pobyt w hotelu Hilton w Nowym Jorku, proszę podać koszty lotu i hotelu.


# Inicjalizacja usługi Azure AI Agent oraz pobranie informacji konfiguracyjnych z **.env**

### **.env**

Utwórz plik .env

**.env** zawiera łańcuch połączenia z usługą Azure AI Agent, model używany przez AOAI oraz odpowiednią usługę API Google API Search, ENDPOINT itp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nazwa wdrożenia modelu usługi Azure AI Agent"

[**UWAGA**] Będziesz potrzebować modelu z limitem 100 000 Rate Limit (Tokenów na minutę) oraz limitem 600 (Żądań na minutę)

  Model można uzyskać w Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Twój łańcuch połączenia projektu usługi Azure AI Agent"

  Łańcuch połączenia projektu można znaleźć w przeglądzie projektu na ekranie portalu AI Foundry.

- **SERPAPI_SEARCH_API_KEY** = "Twój klucz API wyszukiwania SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Twój punkt końcowy SERPAPI wyszukiwania"

Aby uzyskać nazwę wdrożenia modelu i łańcuch połączenia projektu usługi Azure AI Agent, musisz utworzyć usługę Azure AI Agent. Zaleca się użycie [tego szablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) do jej bezpośredniego utworzenia (***Uwaga:*** Usługa Azure AI Agent jest obecnie dostępna tylko w ograniczonym regionie. Zaleca się, abyś zapoznał się z [tym linkiem](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) w celu ustawienia regionu).

Agent musi mieć dostęp do SERPAPI. Zaleca się rejestrację za pomocą [tego linku](https://serpapi.com/searches). Po rejestracji możesz uzyskać unikalny klucz API i punkt końcowy.


# Setup 

Aby uruchomić ten notatnik, musisz upewnić się, że zainstalowałeś wymagane biblioteki, uruchamiając `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Twoja wersja Semantic Kernel powinna być co najmniej 1.27.2.


Załaduj ustawienia i zasoby z pliku .env, upewnij się, że dodałeś swoje klucze i ustawienia oraz utworzyłeś lokalny plik .env.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Zaloguj się do Azure

Teraz musisz zalogować się do Azure. Otwórz terminal i uruchom następujące polecenie:

```bash
az login
```

To polecenie poprosi Cię o wprowadzenie danych logowania do Azure, co umożliwi poprawne działanie usługi Azure AI Agent.


# Explanation:
To jest zmienna przechowująca klucz API do uzyskania dostępu do usługi SERP (Search Engine Results Page). Klucz API to unikalny identyfikator używany do uwierzytelniania żądań powiązanych z Twoim kontem.

Purpose: Celem tego wiersza jest przechowanie klucza API w zmiennej, aby można było go użyć do uwierzytelniania żądań do usługi SERP API. Klucz API jest wymagany, aby uzyskać dostęp do usługi i wykonywać wyszukiwania.
How to Get a SERP API Key: Aby uzyskać klucz API SERP, wykonaj następujące ogólne kroki na https://serpapi.com (dokładne kroki mogą się różnić w zależności od konkretnej usługi SERP API, której używasz):

Choose a SERP API Service: Istnieje kilka usług SERP API, takich jak SerpAPI, Google Custom Search JSON API i inne. Wybierz tę, która najlepiej odpowiada Twoim potrzebom.

Sign Up for an Account: Wejdź na stronę wybranej usługi SERP API i zarejestruj konto. Może być konieczne podanie podstawowych informacji i potwierdzenie adresu e-mail.

Create an API Key: Po rejestracji zaloguj się na swoje konto i przejdź do sekcji API lub panelu sterowania. Znajdź opcję tworzenia lub generowania nowego klucza API.
Copy the API Key to your .env file.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: To zmienna, która przechowuje podstawowy URL dla punktu końcowego API SERP. Nazwa zmiennej BASE_URL jest konwencją używaną do wskazania, że ten URL jest punktem wyjścia do wykonywania zapytań API.
'https://serpapi.com/search':

To jest rzeczywisty ciąg znaków URL przypisany do zmiennej BASE_URL. Reprezentuje punkt końcowy do wykonywania zapytań wyszukiwania za pomocą API SERP.

# Purpose:
Celem tej linii jest zdefiniowanie stałej, która przechowuje podstawowy URL dla API SERP. Ten URL będzie używany jako punkt wyjścia do tworzenia zapytań API w celu wykonywania operacji wyszukiwania.

# Usage:
Definiując podstawowy URL w zmiennej, możesz łatwo ponownie go używać w całym swoim kodzie za każdym razem, gdy potrzebujesz wykonać zapytania do API SERP. Sprawia to, że Twój kod jest bardziej łatwy w utrzymaniu i zmniejsza ryzyko błędów wynikających z umieszczania URL na stałe w wielu miejscach. Obecny przykład to https://serpapi.com/search?engine=bing, który używa API wyszukiwania Bing. Różne API można wybrać na https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Wyjaśnienie:

Tutaj znajduje się kod twojej wtyczki.

Definicja klasy: `class BookingPlugin`: Definiuje klasę o nazwie BookingPlugin, która zawiera metody do rezerwacji hoteli i lotów.

Metoda rezerwacji hotelu:

- `@kernel_function(description="booking hotel")`: Dekorator, który opisuje funkcję jako funkcję jądra do rezerwacji hoteli.
- `def booking_hotel(self, query: Annotated[str, "Nazwa miasta"], check_in_date: Annotated[str, "Data zameldowania w hotelu"], check_out_date: Annotated[str, "Data wymeldowania z hotelu"]) -> Annotated[str, "Zwraca wynik rezerwacji hotelu"]:`: Definiuje metodę do rezerwacji hoteli z adnotowanymi parametrami i typem zwracanym.

Metoda tworzy słownik parametrów dla zapytania o rezerwację hotelu i wysyła żądanie GET do API SERP. Sprawdza status odpowiedzi i zwraca właściwości hotelu, jeśli zapytanie zakończyło się sukcesem, lub None, jeśli żądanie nie powiodło się.

Metoda rezerwacji lotu: 

- `@kernel_function(description="booking flight")`: Dekorator, który opisuje funkcję jako funkcję jądra do rezerwacji lotów.
- `def booking_flight(self, origin: Annotated[str, "Miejsce wylotu"], destination: Annotated[str, "Miejsce docelowe"], outbound_date: Annotated[str, "Data wylotu"], return_date: Annotated[str, "Data powrotu"]) -> Annotated[str, "Zwraca wynik rezerwacji lotu"]:`: Definiuje metodę do rezerwacji lotów z adnotowanymi parametrami i typem zwracanym.

Metoda tworzy słowniki parametrów dla zapytań o lot wylotowy i powrotny, a następnie wysyła żądania GET do API SERP. Sprawdza status odpowiedzi i dołącza informacje o locie do łańcucha wynikowego, jeśli zapytanie zakończyło się sukcesem, lub wypisuje komunikat o błędzie, jeśli żądanie się nie powiodło. Metoda zwraca łańcuch wynikowy zawierający informacje o locie.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Wyjaśnienie:
Importowanie modułów: Importuj niezbędne moduły dla poświadczeń Azure, agenta AI, zawartości wiadomości czatu, roli autora oraz dekoratora funkcji jądra.

Asynchroniczny menedżer kontekstu: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Ustawia asynchroniczny menedżer kontekstu do obsługi poświadczeń Azure i utworzenia klienta agenta AI.

Nazwa agenta i instrukcje:
- `AGENT_NAME = "BookingAgent"`: Definiuje nazwę agenta.
- `AGENT_INSTRUCTIONS = """..."""`: Zapewnia szczegółowe instrukcje dla agenta, jak obsługiwać zapytania dotyczące rezerwacji.

Tworzenie definicji agenta: `agent_definition = await client.agents.create_agent(...)`: Tworzy definicję agenta ze określonym modelem, nazwą oraz instrukcjami.

Tworzenie agenta AzureAI: `agent = AzureAIAgent(...)`: Tworzy agenta AzureAI przy użyciu klienta, definicji agenta oraz zdefiniowanego wtyczki.

Tworzenie wątku: `thread: AzureAIAgentThread | None = None`: Tworzy wątek dla agenta. Nie jest wymagane wcześniejsze tworzenie wątku – jeśli zostanie podana wartość `None`, nowy wątek zostanie utworzony podczas pierwszego wywołania i zwrócony jako część odpowiedzi.

Wejścia użytkownika: `user_inputs = ["..."]`: Definiuje listę wejść użytkownika do przetworzenia przez agenta.

W bloku finally usuń wątek oraz agenta, aby posprzątać zasoby.


# Uwierzytelnianie

Klasa `DefaultAzureCredential` jest częścią Azure SDK dla Pythona. Zapewnia domyślny sposób uwierzytelniania się w usługach Azure. Próbuje uwierzytelnić się za pomocą wielu metod w określonej kolejności, takich jak zmienne środowiskowe, tożsamość zarządzana i poświadczenia Azure CLI.

Operacje asynchroniczne: moduł aio wskazuje, że klasa DefaultAzureCredential obsługuje operacje asynchroniczne. Oznacza to, że można używać jej z asyncio, aby wykonywać nieblokujące żądania uwierzytelniania.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Zastrzeżenie**:  
Niniejszy dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Mimo że dążymy do jak największej dokładności, prosimy mieć na uwadze, że tłumaczenia automatyczne mogą zawierać błędy lub niedokładności. Oryginalny dokument w jego rodzimym języku powinien być uznawany za źródło autorytatywne. W przypadku informacji o krytycznym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia wykonanego przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z korzystania z tego tłumaczenia.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
